# Installation

In [1]:
! pip install torch
! pip install accelerate
! pip install sacremoses
! pip install transformers datasets
! pip install evaluate sacrebleu loralib
! pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Translation

In [3]:
from huggingface_hub import notebook_login
notebook_login()

## Load dataset

In [4]:
from datasets import Dataset, DatasetDict, load_dataset

# Read your text file
with open('/content/drive/MyDrive/en_ar_custom_data.txt', 'r') as file:
    lines = file.readlines()

# Structure of your dataset
dataset_structure = {'translation': [{'ar': line.split('\t')[1].strip(), 'en': line.split('\t')[0].strip()} for line in lines]}

# Split
train_size = int(0.91 * len(lines))
valid_size = len(lines) - train_size
train_dataset = Dataset.from_dict({key: value[:train_size] for key, value in dataset_structure.items()})
valid_dataset = Dataset.from_dict({key: value[-valid_size:] for key, value in dataset_structure.items()})

# Create a DatasetDict for your first dataset
raw_dataset_1 = DatasetDict({'train': train_dataset, 'validation': valid_dataset})

opus = load_dataset("opus100", "ar-en")

# Add all samples from raw_dataset_1 to opus["train"]
opus_train_dataset = Dataset.from_dict({
    'translation': list(raw_dataset_1['train']['translation']) + list(raw_dataset_1['validation']['translation'])+ list(opus['train']['translation'])})

# Use opus["validation"]
opus_valid_dataset = opus['validation']

# Create a new DatasetDict with the merged 'train' and 'validation' splits
merged_dataset = DatasetDict({
    'train': opus_train_dataset,
    'validation': opus_valid_dataset,
})

# Print information about the updated dataset
print(merged_dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1026167
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})


In [5]:
train = merged_dataset["train"].shuffle(seed=42).select(range(60000))
validation = merged_dataset["validation"].shuffle(seed=42).select(range(2000))

## Preprocess

The next step is to load a model & tokenizer to process (ar-en) language pairs:

In [6]:
from transformers import AutoTokenizer
checkpoint = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

In [7]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
print(model)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62802, 512, padding_idx=62801)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62802, 512, padding_idx=62801)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

# Freeze model weights

In [9]:
# Freeze all layers in the encoder
for layer in model.model.encoder.layers:
    for parameter in layer.parameters():
        parameter.requires_grad = False

# Freeze all layers in the decoder
for layer in model.model.decoder.layers:
    for parameter in layer.parameters():
        parameter.requires_grad = False


# Lora Config

In [10]:
from peft import LoraConfig, TaskType, get_peft_model
Lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model_checkpoint = get_peft_model(model, Lora_config)
print(model_checkpoint.print_trainable_parameters())

trainable params: 589,824 || all params: 77,407,232 || trainable%: 0.7619753151746855
None


# preprocess

In [11]:
source_lang = "en"
target_lang = "ar"
prefix = ""

def preprocess_function(examples):
    inputs = [ prefix+example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [12]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_validation = validation.map(preprocess_function, batched=True)

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

## Evaluate

In [14]:
import evaluate

metric = evaluate.load("sacrebleu")

In [15]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [16]:
import accelerate

In [17]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


## Train

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="finetuned_helsinki_peft_model__en_to_ar",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    save_total_limit=3,
    max_steps=10000,
    warmup_steps=500,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)
trainer = Seq2SeqTrainer(
    model=model_checkpoint,# peft model
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss,Validation Loss,Bleu,Gen Len
500,1.943100,1.770475,26.928800,13.456000
1000,1.914900,1.769974,27.582500,13.516000
1500,1.935600,1.768939,26.931400,13.544500
2000,1.924200,1.768111,26.926500,13.458500
2500,1.954200,1.768606,26.974500,13.504500
3000,1.941700,1.767624,27.196600,13.545000
3500,1.923300,1.767508,26.797300,13.522500
4000,1.929000,1.766963,26.880800,13.545500
4500,1.950400,1.766915,27.019200,13.484000
5000,1.914800,1.767314,27.000300,13.522500


TrainOutput(global_step=10000, training_loss=1.9307226989746094, metrics={'train_runtime': 3356.5279, 'train_samples_per_second': 71.502, 'train_steps_per_second': 2.979, 'total_flos': 3595317466890240.0, 'train_loss': 1.9307226989746094, 'epoch': 4.0})

In [22]:
trainer.push_to_hub()

events.out.tfevents.1709935506.cdda3d6febef.1069.3:   0%|          | 0.00/17.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Elalimy/finetuned_helsinki_peft_model__en_to_ar/commit/36aed80b8852121d0cc8a12f0e921f0700290b07', commit_message='End of training', commit_description='', oid='36aed80b8852121d0cc8a12f0e921f0700290b07', pr_url=None, pr_revision=None, pr_num=None)

## Inference

In [23]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer
HUGGING_FACE_USER_NAME = "elalimy"
model_name = "finetuned_helsinki_peft_model__en_to_ar"
peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
AI_model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

In [27]:
def generate_translation(model, tokenizer, source_text, device="cuda"):
    # Encode the source text
    input_ids = tokenizer.encode(source_text, return_tensors='pt').to(device)

    # Move the model to the same device as input_ids
    model = AI_model.to(device)

    # Generate the translation with adjusted decoding parameters
    generated_ids = model.generate(
        input_ids=input_ids,
        max_length=512,  # Adjust max_length if needed
        num_beams=4,
        length_penalty=5,  # Adjust length_penalty if needed
        no_repeat_ngram_size=4,
        early_stopping=True
    )

    # Decode the generated translation excluding special tokens
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    print("Source text:", source_text)
    print("Generated translation:", generated_text)

# Example usage
source_text = "This method for computation instead of brute force is a technique known as dynamic programming.You first solve the smallest subproblem first and then reusing that result you solve the next biggest subproblem, saving that result, reusing it again, and so on. This is exactly what you did by populating each cell from the top right to the bottom left. It’s a well-known technique in computer science! "
generate_translation(model, tokenizer, source_text)


Source text: This method for computation instead of brute force is a technique known as dynamic programming. You first solve the smallest subproblem first and then reusing that result you solve the next biggest subproblem, saving that result, reusing it again, and so on. This is exactly what you did by populating each cell from the top right to the bottom left. It’s a well-known technique in computer science! 
Generated translation: هذه الطريقة للحساب بدلاً من القوة الغاشمة هي تقنية معروفة باسم البرمجة الدينامية. أولاً تحل المشكلة الفرعية الأصغر أولاً ثم تعيد استخدام تلك النتيجة في حل المشكلة التالية الأكبر، حفظ النتيجة، إعادة استخدامها مرة أخرى، وهكذا. هذا بالضبط ما قمت به عن طريق تعبئة كل خلية من الأعلى إلى الأسفل الأيسر. إنها تقنية معروفة جيداً في علوم الحاسوب!


In [25]:
trainer.evaluate()

{'eval_loss': 1.765509009361267,
 'eval_bleu': 26.7751,
 'eval_gen_len': 13.524,
 'eval_runtime': 113.4984,
 'eval_samples_per_second': 17.621,
 'eval_steps_per_second': 0.74,
 'epoch': 4.0}

In [26]:
drive_folder = '/content/drive/MyDrive/english_arabic_model'
model.save_pretrained(drive_folder)
tokenizer.save_pretrained(drive_folder)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}


('/content/drive/MyDrive/english_arabic_model/tokenizer_config.json',
 '/content/drive/MyDrive/english_arabic_model/special_tokens_map.json',
 '/content/drive/MyDrive/english_arabic_model/vocab.json',
 '/content/drive/MyDrive/english_arabic_model/source.spm',
 '/content/drive/MyDrive/english_arabic_model/target.spm',
 '/content/drive/MyDrive/english_arabic_model/added_tokens.json')